In [5]:
# convert scw.list into pointing array

import numpy as np
import astropy.io.fits as fits
from astropy.table import Table
import astropy.time as at
from datetime import datetime
from PointingClusters import save_clusters
from IntegralPointingClustering import ClusteredQuery


with open("scw.list","r") as scw_file:
    scws = []
    with fits.open("crab_data/1667/pointing.fits") as file:
        tab = Table.read(file[1])
        for line in scw_file:
            scw = line[9:21]
            pointing_fits_info = tab[tab["PTID_ISOC"]==scw[:8]]
            t = at.Time(f'{pointing_fits_info["TSTART"][0]+2451544.5}', format='jd').iso[:-4]
            y, m, d, h, mi, s = int(t[:4]), int(t[5:7]), int(t[8:10]), int(t[11:13]), int(t[14:16]), int(t[17:19])
            tstart = datetime(y, m, d, h, mi, s)
            rax, decx= pointing_fits_info["RA_SPIX", "DEC_SPIX"].as_array()[0]
            
            scws.append(np.array([scw, rax, decx, tstart]))
            
    scws = np.array(scws)
    
cq = ClusteredQuery(
    scws,
    angle_weight=0.,
    time_weight=5.,
    max_distance=1.,
    min_ang_distance=1.5,
    max_ang_distance=7.5,
    cluster_size_range = (2,2),
    failed_improvements_max=3,
    suboptimal_cluster_size=1,
    close_suboptimal_cluster_size=1
).get_clustered_scw_ids()

pointings = []
for size in range(2, 3):
    for cluster in cq[size]:
        pointings.append(tuple([(i, f"crab_data/{i[:4]}") for i in cluster]))
pointings = tuple(pointings)

folder = "orbit_1667_spimodfit_scws"

save_clusters(pointings, folder)


In [ ]:
# quartic root

import matplotlib.pyplot as plt
import math

def quartic_root(A, B, C, D, E):
    alpha = -3 * B**2 / (8 * A**2) + C/A
    beta = B**3 / (8 * A**3) - B * C / (2 * A**2) + D/A
    gamma = -3 * B**4 / (256 * A**4) + C * B**2 / (16 * A**3) - B * D / (4 * A**2) + E / A
    
    if beta == 0.:
        s1 = (alpha**2 - 4*gamma)**0.5
        s2 = ((-alpha + s1) / 2)**0.5
        x = -B/(4*A) + s2
        return x
    
    P = -(alpha**2)/12 - gamma
    Q = -(alpha**3)/108 + alpha*gamma/3 - (beta**2) / 8
    R = -Q/2 + (Q**2 / 4 + P**3 / 27)**0.5
    U = R**(1/3)
    if U == 0.:
        y = -5/6*alpha - (Q**(1/3))
    else:
        y = -5/6*alpha + U - P/(3*U)
    W = (alpha + 2*y)**0.5
    
    s1 = 2*beta/W
    s2 = (-(3*alpha + 2*y + s1))**0.5
    s3 = (W + s2) / 2
    x = -B / (4*A) + s3
    return x

In [ ]:
# quartic root tester

res = []
for i in range(1000000):
    arg = np.random.random(5)*1000-500
    A = float(arg[0])
    B = float(arg[1])
    C = float(arg[2])
    D = float(arg[3])
    E = float(arg[4])
    x = quartic_root(A, B, C, D, E)

            
    res.append(
        (A*x**4 + B*x**3 + C*x**2 + D*x + E)
    )
    if abs(res[-1]) > 1.:
        print(A, B, C, D, E, x, res[-1])
plt.scatter([i.real for i in res], [i.imag for i in res])

In [ ]:
# number of scws in crab FOV per revolulution

from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
from astropy.coordinates import SkyCoord

p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
searchquerry = SearchQuery(position=p, radius="5 degree",)
cat = IntegralQuery(searchquerry)
f = Filter(SCW_TYPE="POINTING")
scw_ids_all = cat.apply_filter_on_revolutions(f)
for orbit, count in scw_ids_all.items():
    print(orbit, count)

In [ ]:
# create fits file with only some rows

from astropy.io import fits
t = fits.open('sec_source_test.fits.gz')
tbdata = t[1].data
mask = [False for i in range(len(tbdata))]
mask[0] = True
mask[1] = True
newtbdata = tbdata[mask]
hdu = fits.BinTableHDU(data=newtbdata, name="SPI.-SRCL-CAT")
hdu.writeto('new_sec_source_test.fits.gz')